In [1]:
import pandas as pd
import math
import matplotlib.pyplot as plt
import folium
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Step 1: Identify neighbours ----> 4km

In [2]:
# Haversine formula 
def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius (km)
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    return R * c

# Load stations 
# Reading only the necessary columns, then keep unique stations
stations = pd.read_csv("cleaned_data.csv", usecols=["station_id","latitude","longitude","station_name"]).drop_duplicates()


# ---- Build adjacency list ----
cutoff_km = 5
adj = {}
for i, s1 in stations.iterrows():
    neigh = []
    for j, s2 in stations.iterrows():
        if s1.station_id == s2.station_id:
            continue
        d = haversine_km(s1.latitude, s1.longitude, s2.latitude, s2.longitude)
        if d <= cutoff_km:
            neigh.append((s2.station_id, round(d,3)))
    adj[s1.station_id] = neigh

# ---- Store neighbour info in dataframe ----
stations["neighbours"] = stations["station_id"].map(lambda sid: [n[0] for n in adj[sid]])
stations["neighbour_count"] = stations["neighbours"].apply(len)

# ---- Export adjacency list ----
edges = []
for sid, neigh_list in adj.items():
    for n_id, dist in neigh_list:
        edges.append({"station_a": sid, "station_b": n_id, "distance_km": dist})

edges_df = pd.DataFrame(edges)
#stations.to_csv("stations_with_neighbours.csv", index=False)
#edges_df.to_csv("station_edges_within4km.csv", index=False)

print("✅ Done — neighbours computed using Haversine (cutoff=4 km)")

✅ Done — neighbours computed using Haversine (cutoff=4 km)


In [3]:
stations

,station_id,latitude,longitude,station_name,neighbours,neighbour_count
0,station 1,0.331768,34.252754,St. Augustine Butunyi Primary school,"[station 15, station 5, station 7]",3
720,station 10,0.444917,34.240333,Nambale Boys High School,[station 22],1
1410,station 11,0.444500,34.332944,Fr. Simon Sibembe Secondary School,"[station 19, station 26]",2
1962,station 12,0.504528,34.391667,Khayo Secondary School,"[station 17, station 2, station 24]",3
2579,station 13,0.398389,34.264028,St. Paul's Mabunge Secondary School,"[station 25, station 4, station 5]",3
3298,station 14,0.473389,34.291556,St. Charles Lwanga Emukhuyu Secondary School,"[station 19, station 26, station 27]",3
3834,station 15,0.313291,34.265972,Busiada Girls,"[station 1, station 23, station 7]",3
4671,station 16,-1.310678,36.812921,Strathmore Nairobi,[],0
5206,station 17,0.479778,34.361500,St. Paul Elwanikha Girls School,"[station 12, station 19, station 2]",3
5926,station 18,0.416700,34.200000,Sikoma,[],0


In [4]:
# Center map on Busia region (roughly around average lat/lon)
m = folium.Map(location=[stations["latitude"].mean(), stations["longitude"].mean()], zoom_start=10)

# Add stations
for _, row in stations.iterrows():
    color = "green" if row["neighbour_count"] > 0 else "red"  # Red for isolated stations
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=5 + row["neighbour_count"],  # bigger if more neighbours
        color="black",
        weight=1,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f"Station: {row['station_id']};<br>Station name: {row['station_name']}; <br>Neighbours: {row['neighbour_count']}"
    ).add_to(m)

# Add edges (neighbour connections)
for _, row in edges_df.iterrows():
    s1 = stations.loc[stations["station_id"] == row["station_a"]].iloc[0]
    s2 = stations.loc[stations["station_id"] == row["station_b"]].iloc[0]
    folium.PolyLine(
        locations=[[s1["latitude"], s1["longitude"]], [s2["latitude"], s2["longitude"]]],
        color="blue", weight=1, opacity=0.5
    ).add_to(m)
m


- Based on the above output, the following stations will be excluded from the study: Station 16, Station 23, Station 24, Station 28, Station 8 and Station 18.

# Step 2: Spatial Outliers

- For each station at each timestamp, check if the z-score of observation-median adjusted by the interquatire range of the neighbourhood is within the threshold

In [5]:
# Loading the full data
full_data=pd.read_csv('cleaned_data.csv', parse_dates=["timestamp"]).drop(columns=["Unnamed: 0"])

# Keep only QC-relevant variables + time diff
qc_vars = [
    "station_id", 
    "station_name",
    "timestamp", 
    "air_temperature", 
    "air_humidity", 
    "pressure", 
    "wind_speed", 
    "wind_direction", 
    "rain_guage",
    "rain_accumulation",
    "time_diff_minutes"
]

data_qc = full_data[qc_vars]
data_qc.head(3)

,station_id,station_name,timestamp,air_temperature,air_humidity,pressure,wind_speed,wind_direction,rain_guage,rain_accumulation,time_diff_minutes
0,station 1,St. Augustine Butunyi Primary school,2025-08-27 13:39:04.080750+03:00,27.6,49.0,87550.0,0.8,204,0.0,318.516,NaN
1,station 1,St. Augustine Butunyi Primary school,2025-08-27 13:54:10.032500+03:00,27.1,52.0,87530.0,0.0,316,0.0,318.516,15.099196
2,station 1,St. Augustine Butunyi Primary school,2025-08-27 14:09:17.133825+03:00,27.1,51.0,87510.0,0.0,76,0.0,318.516,15.118355


In [6]:
# Computing a new column ----> rainfall column
data_qc['rainfall'] = data_qc.groupby('station_id')['rain_accumulation'].diff().fillna(0)

# Ensure no negative rainfall
data_qc['rainfall'] = data_qc['rainfall'].apply(lambda x: max(0, x))
data_qc

,station_id,station_name,timestamp,air_temperature,air_humidity,pressure,wind_speed,wind_direction,rain_guage,rain_accumulation,time_diff_minutes,rainfall
0,station 1,St. Augustine Butunyi Primary school,2025-08-27 13:39:04.080750+03:00,27.6,49.0,87550.0,0.8,204,0.0,318.516,NaN,0.0
1,station 1,St. Augustine Butunyi Primary school,2025-08-27 13:54:10.032500+03:00,27.1,52.0,87530.0,0.0,316,0.0,318.516,15.099196,0.0
2,station 1,St. Augustine Butunyi Primary school,2025-08-27 14:09:17.133825+03:00,27.1,51.0,87510.0,0.0,76,0.0,318.516,15.118355,0.0
3,station 1,St. Augustine Butunyi Primary school,2025-08-27 14:24:22.665273+03:00,26.9,54.0,87490.0,0.0,212,0.0,318.516,15.092191,0.0
4,station 1,St. Augustine Butunyi Primary school,2025-08-27 14:39:29.059555+03:00,28.8,47.0,87460.0,0.0,66,0.0,318.516,15.106571,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
19011,station 9,Butula Girls' Secondary School,2025-09-03 12:18:58.753726+03:00,26.0,65.0,87320.0,1.7,234,0.0,434.086,0.251328,0.0
19012,station 9,Butula Girls' Secondary School,2025-09-03 12:34:05.168468+03:00,27.3,61.0,87290.0,1.0,128,0.0,434.086,15.106912,0.0
19013,station 9,Butula Girls' Secondary School,2025-09-03 12:49:11.609516+03:00,26.9,62.0,87280.0,1.0,228,0.0,434.086,15.107351,0.0
19014,station 9,Butula Girls' Secondary School,2025-09-03 13:04:18.019927+03:00,26.6,62.0,87260.0,1.9,294,0.0,434.086,15.106840,0.0


In [7]:
# Neigbours data
# Expand neighbours list into separate rows
neighbours_expanded = (stations.explode("neighbours")[["station_id", "neighbours"]].rename(columns={"neighbours": "neighbour_id"}).dropna())
neighbours_expanded

,station_id,neighbour_id
0,station 1,station 15
0,station 1,station 5
0,station 1,station 7
720,station 10,station 22
1410,station 11,station 19
...,...,...
18300,station 9,station 21
18300,station 9,station 23
18300,station 9,station 3
18300,station 9,station 6


- Station 16,28,24,18,8 are not present, since they had null values. They need to be excluded in the full dataset.

In [8]:
# stations to exclude
invalid_stations = ["station 16", "station 8", "station 28", 
                    "station 24", "station 18"]

# Clean the full dataset
data_qc = data_qc[~data_qc["station_id"].isin(invalid_stations)].reset_index(drop=True)

print("Dropped invalid stations. Remaining stations:", data_qc["station_id"].nunique())

Dropped invalid stations. Remaining stations: 23


In [9]:
# Merging the two datasets

# Some stations don’t record at exactly the same second, but this method
#  lets you align them within ±15 minutes.

# Ensure timestamp is datetime
data_qc["timestamp"] = pd.to_datetime(data_qc["timestamp"])

# Sort for merge_asof
data_qc = data_qc.sort_values("timestamp")

merged = []
for sid, neighs in neighbours_expanded.groupby("station_id"):
    # Extract base station data
    base = data_qc[data_qc["station_id"] == sid] 
    
    #For each neighbour station, filter its rows
    for neigh in neighs["neighbour_id"]:
        neigh_data = data_qc[data_qc["station_id"] == neigh].sort_values("timestamp")
        tmp = pd.merge_asof(
            base,
            neigh_data,
            on="timestamp",
            by=None,  # not same station_id
            direction="nearest",
            tolerance=pd.Timedelta("15min"),
            suffixes=("", "_neigh")
        )
        tmp["neighbour_id"] = neigh
        merged.append(tmp)

merged = pd.concat(merged, ignore_index=True)
merged


,station_id,station_name,timestamp,air_temperature,air_humidity,pressure,wind_speed,wind_direction,rain_guage,rain_accumulation,...,air_temperature_neigh,air_humidity_neigh,pressure_neigh,wind_speed_neigh,wind_direction_neigh,rain_guage_neigh,rain_accumulation_neigh,time_diff_minutes_neigh,rainfall_neigh,neighbour_id
0,station 1,St. Augustine Butunyi Primary school,2025-08-27 13:39:04.080750+03:00,27.6,49.0,87550.0,0.8,204,0.0,318.516,...,28.6,52.0,87510.0,0.0,186.0,0.0,376.428,15.027620,0.0,station 15
1,station 1,St. Augustine Butunyi Primary school,2025-08-27 13:54:10.032500+03:00,27.1,52.0,87530.0,0.0,316,0.0,318.516,...,26.8,52.0,87500.0,0.0,228.0,0.0,376.428,15.106361,0.0,station 15
2,station 1,St. Augustine Butunyi Primary school,2025-08-27 14:09:17.133825+03:00,27.1,51.0,87510.0,0.0,76,0.0,318.516,...,27.9,52.0,87480.0,0.0,204.0,0.0,376.428,15.109078,0.0,station 15
3,station 1,St. Augustine Butunyi Primary school,2025-08-27 14:24:22.665273+03:00,26.9,54.0,87490.0,0.0,212,0.0,318.516,...,27.1,53.0,87460.0,0.0,210.0,0.0,376.428,15.120738,0.0,station 15
4,station 1,St. Augustine Butunyi Primary school,2025-08-27 14:39:29.059555+03:00,28.8,47.0,87460.0,0.0,66,0.0,318.516,...,27.1,53.0,87460.0,0.0,210.0,0.0,376.428,0.309689,0.0,station 15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49474,station 9,Butula Girls' Secondary School,2025-09-03 12:18:58.753726+03:00,26.0,65.0,87320.0,1.7,234,0.0,434.086,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,station 8
49475,station 9,Butula Girls' Secondary School,2025-09-03 12:34:05.168468+03:00,27.3,61.0,87290.0,1.0,128,0.0,434.086,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,station 8
49476,station 9,Butula Girls' Secondary School,2025-09-03 12:49:11.609516+03:00,26.9,62.0,87280.0,1.0,228,0.0,434.086,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,station 8
49477,station 9,Butula Girls' Secondary School,2025-09-03 13:04:18.019927+03:00,26.6,62.0,87260.0,1.9,294,0.0,434.086,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,station 8


In [10]:
# Spatial outliers with description only
def compute_spatial_flags_allvars(merged, variables=None):
    if variables is None:
        variables = [
            "air_temperature",
            "air_humidity",
            "pressure",
            "wind_speed",
            "wind_direction",
            "rainfall"
        ]
    
    results = []

    for (sid, ts), group in merged.groupby(["station_id", "timestamp"]):
        row_result = {
            "station_id": sid,
            "timestamp": ts
        }

        for var in variables:
            x_it = group.iloc[0][var]

            # Neighbour values
            neigh_vals = group[f"{var}_neigh"].dropna().values
            neigh_list = neigh_vals.tolist() if len(neigh_vals) > 0 else []
            
            if len(neigh_vals) < 2:
                flag = -1
                Tit = np.nan
                M_t, IQR = np.nan, np.nan
                description = "insufficient neighbours"
            else:
                # Median + IQR
                M_t = np.median(neigh_vals)
                q25, q75 = np.percentile(neigh_vals, [25, 75])
                IQR = q75 - q25

                if IQR == 0:
                    flag = -1
                    Tit = np.nan
                    description = "IQR=0 (cannot compute standardized difference)"
                else:
                    Tit = (x_it - M_t) / IQR
                    if abs(Tit) > 2.0:
                        flag = 2
                        description = "outlier"
                    elif abs(Tit) > 1.0:
                        flag = 1
                        description = "warning"
                    else:
                        flag = 0
                        description = "normal"

            # Save outputs for this variable
            row_result[f"{var}_value"] = x_it
            row_result[f"{var}_neighs"] = neigh_list
            row_result[f"{var}_median"] = M_t
            row_result[f"{var}_IQR"] = IQR
            row_result[f"{var}_Tit"] = Tit
            row_result[f"{var}_flag"] = flag
            row_result[f"{var}_description"] = description

        results.append(row_result)

    return pd.DataFrame(results)

# Run QC
qc_results = compute_spatial_flags_allvars(merged)

# Example display
qc_results.head()


,station_id,timestamp,air_temperature_value,air_temperature_neighs,air_temperature_median,air_temperature_IQR,air_temperature_Tit,air_temperature_flag,air_temperature_description,air_humidity_value,...,wind_direction_Tit,wind_direction_flag,wind_direction_description,rainfall_value,rainfall_neighs,rainfall_median,rainfall_IQR,rainfall_Tit,rainfall_flag,rainfall_description
0,station 1,2025-08-27 13:39:04.080750+03:00,27.6,"[28.6, 27.4, 28.1]",28.1,0.60,-0.833333,0,normal,49.0,...,-0.235294,0,normal,0.0,"[0.0, 0.0, 0.0]",0.0,0.0,NaN,-1,IQR=0 (cannot compute standardized difference)
1,station 1,2025-08-27 13:54:10.032500+03:00,27.1,"[26.8, 27.6, 28.6]",27.6,0.90,-0.555556,0,normal,52.0,...,0.033058,0,normal,0.0,"[0.0, 0.0, 0.0]",0.0,0.0,NaN,-1,IQR=0 (cannot compute standardized difference)
2,station 1,2025-08-27 14:09:17.133825+03:00,27.1,"[27.9, 27.4, 27.5]",27.5,0.25,-1.600000,1,warning,51.0,...,-5.485714,2,outlier,0.0,"[0.0, 0.0, 0.0]",0.0,0.0,NaN,-1,IQR=0 (cannot compute standardized difference)
3,station 1,2025-08-27 14:24:22.665273+03:00,26.9,"[27.1, 27.7, 26.9]",27.1,0.40,-0.500000,0,normal,54.0,...,0.028571,0,normal,0.0,"[0.0, 0.0, 0.0]",0.0,0.0,NaN,-1,IQR=0 (cannot compute standardized difference)
4,station 1,2025-08-27 14:39:29.059555+03:00,28.8,"[27.1, 27.4, 28.8]",27.4,0.85,1.647059,1,warning,47.0,...,-6.233333,2,outlier,0.0,"[0.0, 0.0, 0.0]",0.0,0.0,NaN,-1,IQR=0 (cannot compute standardized difference)


In [11]:
qc_results.to_csv('spatial_outliers_5km.csv')

In [12]:
# List of variables
variables = [
    "air_temperature",
    "air_humidity",
    "pressure",
    "wind_speed",
    "wind_direction",
    "rainfall"
]

# Initialize summary list
summary_list = []

for var in variables:
    summary = (qc_results
               .groupby("station_id")[f"{var}_flag"]
               .apply(lambda x: (x == 2).sum())  # count only outliers
               .reset_index(name=f"{var}_outlier_count")
              )
    summary_list.append(summary)

# Merge all variable summaries into one DataFrame
from functools import reduce
spatial_summary = reduce(lambda left, right: pd.merge(left, right, on="station_id"), summary_list)

# Sort by total outliers across all variables
spatial_summary['total_outliers'] = spatial_summary[[f"{var}_outlier_count" for var in variables]].sum(axis=1)
spatial_summary = spatial_summary.sort_values('total_outliers', ascending=False)

spatial_summary


,station_id,air_temperature_outlier_count,air_humidity_outlier_count,pressure_outlier_count,wind_speed_outlier_count,wind_direction_outlier_count,rainfall_outlier_count,total_outliers
20,station 6,243,323,718,64,221,6,1575
18,station 4,246,351,680,64,159,8,1508
9,station 2,262,221,664,69,258,1,1475
10,station 20,249,175,679,81,206,4,1394
21,station 7,114,168,701,61,165,9,1218
13,station 23,206,196,294,81,152,14,943
15,station 26,114,117,539,51,112,4,937
14,station 25,221,208,1,70,362,9,871
3,station 12,265,168,0,88,268,7,796
6,station 15,221,223,0,66,189,7,706


In [13]:
spatial_summary.to_csv('outliers_summary_5km.csv')

# Step 3: Rainfall – Faulty Zeros (FZ)

- For each station with a rainfall reading of zero, and timestamp compare median neighbourhood rainfall (minimum number of stations for median = 5) for the same timestamp

In [14]:
# Faulty zeroes with detailed logic and separated -1 cases
def compute_fz_with_median_final(merged, min_neigh=2, consecutive_threshold=2):
    """
    Compute Faulty Zero (FZ_Rain) QC flags with explanations.

    Parameters:
    - min_neigh: minimum number of neighbours required (default 2)
    - consecutive_threshold: consecutive timestamps of neighbour_median>0 while 
      station rainfall==0 to trigger FZ=2 (default 2)
       
    FZ_flag meanings: 
      -1 = insufficient neighbours OR time interval > 15 min
       0 = normal
       1 = faulty non-zero (station > 0, neighbours = 0)
       2 = faulty zero (station = 0, neighbours > 0 for ≥ threshold consecutive steps)
    """

    merged = merged.copy()
    merged['timestamp'] = pd.to_datetime(merged['timestamp'])

    # aggregate neighbour rainfall into a list per station,timestamp
    neigh_agg = (merged
                 .groupby(['station_id','timestamp'])['rainfall_neigh']
                 .apply(list)
                 .reset_index(name='rainfall_neigh_list'))

    # target rainfall
    target = (merged
              .groupby(['station_id','timestamp'])['rainfall']
              .first()
              .reset_index())

    # merge to one row per station,timestamp
    qc_df = target.merge(neigh_agg, on=['station_id','timestamp'], how='left')
    qc_df = qc_df.sort_values(['station_id','timestamp']).reset_index(drop=True)

    # compute neighbour median
    qc_df['rainfall_neigh_median'] = qc_df['rainfall_neigh_list'].apply(
        lambda x: np.median([v for v in x if not pd.isna(v)]) if isinstance(x, list) and any(not pd.isna(v) for v in x) else np.nan
    )

    # initialize outputs and trackers
    qc_df['FZ_flag'] = 0
    qc_df['FZ_description'] = "normal"
    qc_df['logic'] = ""
    streak_tracker = {sid: 0 for sid in qc_df['station_id'].unique()}
    fz_active = {sid: False for sid in qc_df['station_id'].unique()}

    # iterate in time order
    for idx, row in qc_df.iterrows():
        sid = row['station_id']
        station_val = row['rainfall']
        neigh_vals = row['rainfall_neigh_list']
        neigh_median = row['rainfall_neigh_median']

        # --- Case 1: Time interval > 15 min (any NaN in neighbour list) ---
        if isinstance(neigh_vals, list) and any(pd.isna(v) for v in neigh_vals):
            qc_df.at[idx, 'FZ_flag'] = -1
            qc_df.at[idx, 'FZ_description'] = "time interval > 15 minutes"
            qc_df.at[idx, 'logic'] = f"Neighbour values contain NaN ; FZ_flag = -1"
            streak_tracker[sid] = 0
            fz_active[sid] = False
            continue

        # --- Case 2: Insufficient neighbours ---
        if not isinstance(neigh_vals, list) or len(neigh_vals) < min_neigh:
            qc_df.at[idx, 'FZ_flag'] = -1
            qc_df.at[idx, 'FZ_description'] = "insufficient neighbours"
            qc_df.at[idx, 'logic'] = f"Only {0 if neigh_vals is None else len(neigh_vals)} neighbours ; FZ_flag = -1"
            streak_tracker[sid] = 0
            fz_active[sid] = False
            continue

        # --- Station rainfall = 0 ---
        if station_val == 0:
            if neigh_median > 0:
                streak_tracker[sid] += 1
            else:
                streak_tracker[sid] = 0
                fz_active[sid] = False

            if streak_tracker[sid] >= consecutive_threshold or fz_active[sid]:
                qc_df.at[idx, 'FZ_flag'] = 2
                qc_df.at[idx, 'FZ_description'] = "faulty zero"
                fz_active[sid] = True
                qc_df.at[idx, 'logic'] = f"Station = 0, Neighbour median = {neigh_median:.2f}, streak >= {consecutive_threshold} ; FZ_flag = 2"
            else:
                qc_df.at[idx, 'FZ_flag'] = 0
                qc_df.at[idx, 'FZ_description'] = "normal"
                qc_df.at[idx, 'logic'] = f"Station = 0, Neighbour median = {neigh_median:.2f} ; normal"

        # --- Station rainfall > 0 ---
        else:
            streak_tracker[sid] = 0
            fz_active[sid] = False
            if neigh_median == 0:
                qc_df.at[idx, 'FZ_flag'] = 1
                qc_df.at[idx, 'FZ_description'] = "faulty non-zero"
                qc_df.at[idx, 'logic'] = f"Station = {station_val} > 0, Neighbour median = 0 ; FZ_flag = 1"
            else:
                qc_df.at[idx, 'FZ_flag'] = 0
                qc_df.at[idx, 'FZ_description'] = "normal"
                qc_df.at[idx, 'logic'] = f"Station = {station_val} > 0, Neighbour median = {neigh_median:.2f} ; normal"

    return qc_df


# Example usage
qc_results_fz = compute_fz_with_median_final(merged)
qc_results_fz[['station_id','timestamp','rainfall',
               'rainfall_neigh_list','rainfall_neigh_median',
               'FZ_flag','FZ_description','logic']].head(30)


,station_id,timestamp,rainfall,rainfall_neigh_list,rainfall_neigh_median,FZ_flag,FZ_description,logic
0,station 1,2025-08-27 13:39:04.080750+03:00,0.0,"[0.0, 0.0, 0.0]",0.0,0,normal,"Station = 0, Neighbour median = 0.00 ; normal"
1,station 1,2025-08-27 13:54:10.032500+03:00,0.0,"[0.0, 0.0, 0.0]",0.0,0,normal,"Station = 0, Neighbour median = 0.00 ; normal"
2,station 1,2025-08-27 14:09:17.133825+03:00,0.0,"[0.0, 0.0, 0.0]",0.0,0,normal,"Station = 0, Neighbour median = 0.00 ; normal"
3,station 1,2025-08-27 14:24:22.665273+03:00,0.0,"[0.0, 0.0, 0.0]",0.0,0,normal,"Station = 0, Neighbour median = 0.00 ; normal"
4,station 1,2025-08-27 14:39:29.059555+03:00,0.0,"[0.0, 0.0, 0.0]",0.0,0,normal,"Station = 0, Neighbour median = 0.00 ; normal"
5,station 1,2025-08-27 14:54:35.239323+03:00,0.0,"[0.0, 0.0, 0.0]",0.0,0,normal,"Station = 0, Neighbour median = 0.00 ; normal"
6,station 1,2025-08-27 15:09:41.712196+03:00,0.0,"[0.0, 0.0, 0.0]",0.0,0,normal,"Station = 0, Neighbour median = 0.00 ; normal"
7,station 1,2025-08-27 15:25:03.676310+03:00,0.0,"[0.0, 0.0, 0.0]",0.0,0,normal,"Station = 0, Neighbour median = 0.00 ; normal"
8,station 1,2025-08-27 15:40:09.819732+03:00,0.0,"[0.0, 0.0, 0.0]",0.0,0,normal,"Station = 0, Neighbour median = 0.00 ; normal"
9,station 1,2025-08-27 15:55:16.212109+03:00,0.0,"[0.0, 0.0, 0.0]",0.0,0,normal,"Station = 0, Neighbour median = 0.00 ; normal"


In [15]:
qc_results_fz.to_csv('faulty_zero_5km.csv')

In [16]:
# summary of faulty zeroes per station
fz_summary = (qc_results_fz
              .groupby('station_id')['FZ_flag']
              .apply(lambda x: (x == 2).sum())   # count only FZ=2
              .reset_index(name='faulty_zero_count')
              .sort_values('faulty_zero_count', ascending=False))

fz_summary

,station_id,faulty_zero_count
6,station 15,7
2,station 11,6
19,station 5,6
0,station 1,4
14,station 25,3
10,station 20,3
13,station 23,3
21,station 7,3
11,station 21,3
9,station 2,2


In [17]:
fz_summary.to_csv('faulty_zero_summary_5km.csv')

# Step 4: High Rainfall

In [18]:
# High rainfall flagging with explicit logic and separated -1 cases
def compute_hi_with_median(merged, min_neigh=2, static_threshold=10, variable_factor=4, median_threshold=0.4):
    """
    Compute High Rainfall (HI_Rain) QC flags with detailed logic explanation.
    -1 now separated into:
        - insufficient neighbours
        - time interval > 15 minutes (NaN in neighbour values)
    """

    merged = merged.copy()
    merged['timestamp'] = pd.to_datetime(merged['timestamp'])

    # Aggregate neighbour rainfall into a list
    neigh_agg = (merged
                 .groupby(['station_id','timestamp'])['rainfall_neigh']
                 .apply(list)
                 .reset_index(name='rainfall_neigh_list'))

    # Target rainfall
    target = (merged
              .groupby(['station_id','timestamp'])['rainfall']
              .first()
              .reset_index())

    # Merge to get one row per station/timestamp
    qc_df = target.merge(neigh_agg, on=['station_id','timestamp'], how='left')
    qc_df = qc_df.sort_values(['station_id','timestamp']).reset_index(drop=True)

    # Compute median of neighbours (ignore NaNs)
    qc_df['rainfall_neigh_median'] = qc_df['rainfall_neigh_list'].apply(
        lambda x: np.median([v for v in x if not pd.isna(v)]) if isinstance(x, list) and any(not pd.isna(v) for v in x) else np.nan
    )

    # Flagging logic
    def flag_hi(row):
        neigh_vals = row['rainfall_neigh_list']
        station_rain = row['rainfall']

        # --- Time interval > 15 minutes ---
        if isinstance(neigh_vals, list) and any(pd.isna(v) for v in neigh_vals):
            return -1, "time interval > 15 minutes"

        # --- Insufficient neighbours ---
        if neigh_vals is None or len(neigh_vals) < min_neigh:
            return -1, "insufficient neighbours"

        neigh_median = row['rainfall_neigh_median']

        # Case A: dry neighbourhood
        if neigh_median < median_threshold:
            flag = 2 if station_rain > static_threshold else 0
        # Case B: wet neighbourhood
        else:
            dynamic_threshold = variable_factor * neigh_median
            flag = 2 if station_rain > dynamic_threshold else 0

        description = "high rainfall outlier" if flag == 2 else "normal"
        return flag, description

    qc_df[['HI_flag', 'HI_description']] = qc_df.apply(lambda row: pd.Series(flag_hi(row)), axis=1)

    # Detailed logic explanation
    def describe_flag_detailed(row):
        neigh_vals = row['rainfall_neigh_list']
        station_rain = row['rainfall']
        neigh_median = row['rainfall_neigh_median']
        flag = row['HI_flag']

        # --- Time interval > 15 minutes ---
        if isinstance(neigh_vals, list) and any(pd.isna(v) for v in neigh_vals):
            return f"Neighbour values contain NaN ; HI_flag = -1"

        # --- Insufficient neighbours ---
        if neigh_vals is None or len(neigh_vals) < min_neigh:
            return f"Neighbour requirement not met (found {0 if neigh_vals is None else len(neigh_vals)}) → HI_flag = -1"

        # --- Case A: dry neighbourhood ---
        if neigh_median < median_threshold:
            if station_rain > static_threshold:
                return f"Neighbour median: {neigh_median:.2f} < {median_threshold} ; station rainfall: {station_rain} > static threshold: {static_threshold} ; HI_flag = 2"
            else:
                return f"Neighbour median: {neigh_median:.2f} < {median_threshold} ; station rainfall: {station_rain} <= static threshold: {static_threshold} ; HI_flag = 0 (normal)"

        # --- Case B: wet neighbourhood ---
        dynamic_threshold = variable_factor * neigh_median
        if station_rain > dynamic_threshold:
            return f"Neighbour median: {neigh_median:.2f} >= {median_threshold} ; station rainfall: {station_rain} > dynamic threshold: {variable_factor}*median={dynamic_threshold:.2f} ; HI_flag = 2"
        else:
            return f"Neighbour median: {neigh_median:.2f} >= {median_threshold} ; station rainfall: {station_rain} <= dynamic threshold: {variable_factor}*median={dynamic_threshold:.2f} ; HI_flag = 0 (normal)"

    qc_df['logic'] = qc_df.apply(describe_flag_detailed, axis=1)

    return qc_df

# Example usage
qc_results_hi = compute_hi_with_median(merged)
qc_results_hi[['station_id','timestamp','rainfall','rainfall_neigh_list',
               'rainfall_neigh_median','HI_flag','HI_description','logic']].head(20)


,station_id,timestamp,rainfall,rainfall_neigh_list,rainfall_neigh_median,HI_flag,HI_description,logic
0,station 1,2025-08-27 13:39:04.080750+03:00,0.0,"[0.0, 0.0, 0.0]",0.0,0,normal,Neighbour median: 0.00 < 0.4 ; station rainfal...
1,station 1,2025-08-27 13:54:10.032500+03:00,0.0,"[0.0, 0.0, 0.0]",0.0,0,normal,Neighbour median: 0.00 < 0.4 ; station rainfal...
2,station 1,2025-08-27 14:09:17.133825+03:00,0.0,"[0.0, 0.0, 0.0]",0.0,0,normal,Neighbour median: 0.00 < 0.4 ; station rainfal...
3,station 1,2025-08-27 14:24:22.665273+03:00,0.0,"[0.0, 0.0, 0.0]",0.0,0,normal,Neighbour median: 0.00 < 0.4 ; station rainfal...
4,station 1,2025-08-27 14:39:29.059555+03:00,0.0,"[0.0, 0.0, 0.0]",0.0,0,normal,Neighbour median: 0.00 < 0.4 ; station rainfal...
5,station 1,2025-08-27 14:54:35.239323+03:00,0.0,"[0.0, 0.0, 0.0]",0.0,0,normal,Neighbour median: 0.00 < 0.4 ; station rainfal...
6,station 1,2025-08-27 15:09:41.712196+03:00,0.0,"[0.0, 0.0, 0.0]",0.0,0,normal,Neighbour median: 0.00 < 0.4 ; station rainfal...
7,station 1,2025-08-27 15:25:03.676310+03:00,0.0,"[0.0, 0.0, 0.0]",0.0,0,normal,Neighbour median: 0.00 < 0.4 ; station rainfal...
8,station 1,2025-08-27 15:40:09.819732+03:00,0.0,"[0.0, 0.0, 0.0]",0.0,0,normal,Neighbour median: 0.00 < 0.4 ; station rainfal...
9,station 1,2025-08-27 15:55:16.212109+03:00,0.0,"[0.0, 0.0, 0.0]",0.0,0,normal,Neighbour median: 0.00 < 0.4 ; station rainfal...


In [19]:
qc_results_hi.to_csv('high_rainfall_5km.csv')

In [20]:
# summary of high rainfall flags per station
hi_summary = (qc_results_hi
              .groupby('station_id')['HI_flag']
              .apply(lambda x: (x == 2).sum())   # count only HI_flag = 2
              .reset_index(name='high_rainfall_count')
              .sort_values('high_rainfall_count', ascending=False))

hi_summary

,station_id,high_rainfall_count
18,station 4,2
11,station 21,1
16,station 27,1
17,station 3,1
21,station 7,1
1,station 10,0
0,station 1,0
6,station 15,0
5,station 14,0
4,station 13,0


In [21]:
hi_summary.to_csv("high_rainfall_summary_5km.csv")